In [1]:
import pandas as pd
import datetime as datetime
import numpy as np
import copy
from RSU_functions import * 

In [2]:
# If you want to change your marginal income tax (above 72k of taxable income, it is at 41%). Not really needed
tax_info_dict = {'TMI_IR': 0.3, 'cotisation': 0.172, 'flat_tax_plus_value': 0.3, 'deductible_csg':0.0680}

# Put here all sales and vesting event from EquatePlus
please put all event until end of 2022, including sales / vesting from before (other years). If you have sales in 2020 or 2021 either rely on the built in function or please du check yourself how you reported them before. 

If you sold everything you had at one point, before 2021 then you can put only vesting and transaction after this big sale event.

If you sold everything in 2021, please put all information in 2021 and even before if you want the CSG rebate computation, if you don't, you may start with all events starting after this "big sales". Please note this CSG rebate should be prefilled, so it is only to check.

If you did not sell everything, please find out which stock you declared selling in your previous declaration (for stock sold before this year of 2022) and add the amount sold to the vesting event as amount_already_declared

For sales, you need to add the stock unit price. In equate plus, go to the bibliotheque menu, then transaction. For sales, click on the date which is clickable and then report "prix de vente" IN USD (VERY IMPORTANT).

For Macron law, you do not have new macron law 3 stocks from Criteo. Indeed (ridiculously) the date of interest for the macron law 3 to kick in is not the grant date, but the date of the AG autorising the grant. Sadly all of our grants coming before the June 25th 2020 are NOT macron law eligible. Should you decide to have your (wrong) interpretation of the law, it should work though.

If you sold more than 300k this year, please note that the macron law 1 or 2 differs. This notebook is not handling it correctly.

In [3]:

vesting_event = [#each row for a vesting. If you have more vesting, please copy the full line with the comma at the end, and then put your value on each row. 
    # You should fill just the date and the amount. Please let macron_law_id at 0, we do not have newer RSU. If you decide 
    # to have your personnal interpretation of the law, or have non-criteo RSU, please put 3 instead.
                 {'date':datetime.datetime(2020,12,13),'amount':225,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':225},
                {'date':datetime.datetime(2021,3,13),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
                {'date':datetime.datetime(2021,6,15),'amount':29,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':29},
                {'date':datetime.datetime(2021,9,14),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
                {'date':datetime.datetime(2021,12,14),'amount':28,'macron_law_id':0, 'event':'vesting', 'amount_already_declared':28},
    # amount sold is only for vested stock that you sold and declared in the years before. Here the previous stock was sold the 16 of december
    # so it is the last that I am filling. It is not necessary to fill it if you sold everything you had at your last sale event last year
    # here it should therefore do nothing because in this example, all stocks were sold. In case of doubt, just fill it.
    # When filled, these stocks are sold in priority over all else at the next transaction.
    # If I find stocks that were supposedly declared in 2021 that are sold in 2022, I will raise an error and ask you to check what you wrote. 
                {'date':datetime.datetime(2022,3,15),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2022,6,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2022,7,26),'amount':600,'macron_law_id':3, 'event':'vesting'},
                {'date':datetime.datetime(2022,9,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2022,10,25),'amount':75,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2022,12,13),'amount':28,'macron_law_id':0, 'event':'vesting'}
                ]

selling_event = [
                 #each row for a sale. If you have more sale, please copy the full line with the comma at the end, and then put your value on each row. 
    # You should just fill the date, amount, and stock_unit_price_USD, and total_fee_dollars 
    {'date':datetime.datetime(2021,8,3),'amount':282, 'event':'sale', 'stock_unit_price_USD':13.62, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2021,12,16),'amount':56, 'event':'sale', 'stock_unit_price_USD':18.80, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2022,3,16),'amount':28, 'event':'sale', 'stock_unit_price_USD':33.37, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2022,8,6),'amount':628, 'event':'sale', 'stock_unit_price_USD':39.18, 'total_fee_dollars':108.26},
    {'date':datetime.datetime(2022,9,14),'amount':28, 'event':'sale', 'stock_unit_price_USD':36.72, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2022,11,5),'amount':75, 'event':'sale', 'stock_unit_price_USD':40.57, 'total_fee_dollars':38.5},
    {'date':datetime.datetime(2022,12,16),'amount':28, 'event':'sale', 'stock_unit_price_USD':40.71, 'total_fee_dollars':38.5}
]

Computation occurs here, please just run these cells

In [4]:
all_events_sorted = sorted(vesting_event + selling_event, key = lambda k: k['date'])

In [5]:
portfolio = {}
portfolio['available_stock'] = []
portfolio['sale_event_recap'] = []
for event in all_events_sorted:
    if event['event'] == 'vesting':
        portfolio['available_stock'].append(copy.deepcopy(event))
    if event['event'] == 'sale':
        available_stock_after_transaction, sale_event_recap = get_sales_result(event, portfolio,
                                                                               matching_method = get_sale_order_from_optionality)
        portfolio['available_stock'] = available_stock_after_transaction
        portfolio['sale_event_recap'].append(sale_event_recap)
    if event['date'].year == 2022:
        sum_stock_supposed_to_be_sold = 0
        for stock in portfolio['available_stock']:
            sum_stock_supposed_to_be_sold += stock.get('amount_already_declared', 0)
        if sum_stock_supposed_to_be_sold> 0:
            raise Exception(
            '''
            you have stock that you already declared in the previous years that are getting sold in 2022. Please recheck
            your inputs. If you declared them, they should have been sold in the past.            
            ''')
        

In [6]:
available_stock_after_transaction

[]

# Results to put in your taxation returns
Ici, toute les infos sont printer pour remplir le formulaire No 2074. 
pour 511 mettre Action criteo et equateplus.
Vesting amount with moins value, rebate etc... are for debugging purposes, please look if they make sense.

In [7]:
tax = 0
Capital_gain_or_loss = 0
vesting_gain_before_rebate_and_moins_value = 0
rebate_amount = 0
total_selling_price = 0
csg_rebate = 0
Capital_gain_or_loss_2021 = 0
rebate_amount_2021 = 0
vesting_gain_before_rebate_and_moins_value_2021 = 0
for event in portfolio['sale_event_recap']:
    for ev in event:            
        if ev['date de la cession (513)'].year ==2022:
            print('\n ----------------NOUVELLE TRANSACTION ---------- \n')
            for key, value in ev.items():
                print(  key, value)
            tax +=ev['tax']
            Capital_gain_or_loss += ev['resultat']
            rebate_amount += ev['rebate_without_moins_value']
            vesting_gain_before_rebate_and_moins_value += ev['prix daquisition global (521 et 523)']
            total_selling_price += ev['prix de cession net (518)']
            
        if ev['date de la cession (513)'].year ==2021:
            #here it is to compute CSG than can be dedudected from last year, box 6DE. Only what is 
            # subject to IR can be deducted, hence only the  vesting amount after rebate and moins value
            #will be deductible
            Capital_gain_or_loss_2021 += ev['resultat']
            rebate_amount_2021 += ev['rebate_without_moins_value']
            vesting_gain_before_rebate_and_moins_value_2021 += ev['prix daquisition global (521 et 523)']
        



 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2021-03-16 00:00:00
valeur unitaire de la cession (514) 27.980882106322316
nombre de titres cedes (515) 28
montant global (516) 783.4646989770248
frais de cession (517) 32.2824081838001
prix de cession net (518) 751.1822907932246
prix ou valeur acquisition unitaire (520) 27.684563758389263
prix daquisition global (521 et 523) 775.1677852348994
resultat -23.985494441674746
vesting_amount_with_moins_value 775.1677852348994
rebate_without_moins_value 0.0
rebate 0
tax 368.3682687535101

 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2021-08-06 00:00:00
valeur unitaire de la cession (514) 33.18370458202761
nombre de titres cedes (515) 600
montant global (516) 19910.222749216566
frais de cession (517) 45.84568476327602
prix de cession net (518) 19864.37706445329
prix ou valeur acquisition unitaire (520) 36.09909222024263
prix daquisition global (521 et 523) 21659.45533214558
resultat -17

Normally here you should have automatically computed the sum of "plus or moins value", and it should be equal to the value printed in the cell below. Using the overall plus ou moins value, then I will be able to remove it from the vesting price to reduce overall tax burden if negative. 
If there is a rebate, I uniformly apply the rebate to the vesting gain with the moins value.

In [8]:
print('The total capital gain or loss is {}'.format(Capital_gain_or_loss))

The total capital gain or loss is -1580.627325525463


In [9]:
if Capital_gain_or_loss >0:
    vesting_gain_after_rebate_and_moins_value = vesting_gain_before_rebate_and_moins_value - rebate_amount
    total_tax_to_pay_approx = vesting_gain_after_rebate_and_moins_value * (tax_info_dict['TMI_IR'] + tax_info_dict[
            'cotisation']) + Capital_gain_or_loss * tax_info_dict[
                         'flat_tax_plus_value']
    rebate = rebate_amount
else:
    rebate_percent = rebate_amount / vesting_gain_before_rebate_and_moins_value
    vesting_gain_after_rebate_and_moins_value = (vesting_gain_before_rebate_and_moins_value + Capital_gain_or_loss) * (1-rebate_percent)
    rebate = (vesting_gain_before_rebate_and_moins_value + Capital_gain_or_loss) * (rebate_percent)
    total_tax_to_pay_approx = vesting_gain_after_rebate_and_moins_value * (tax_info_dict['TMI_IR'] + tax_info_dict[
            'cotisation'])

In [10]:
if vesting_gain_after_rebate_and_moins_value > 300000:
    print('fill 1TT, form 2042 C with ', vesting_gain_after_rebate_and_moins_value - 300000 )
print('fill 1TZ, form 2042 C with ', vesting_gain_after_rebate_and_moins_value)
print('fill 1UZ , form 2042 C with the rebate in euro', rebate )
print('fill 3VG , form 2042 C with capital gain', max(0,Capital_gain_or_loss) )
print('fill 3VH , form 2042 C with capital loss', 0, 'except if you have other sources of capital losses. You fill 0 because you are removing these losses from the 1TZ field' )


print('You sold for {} this amount of stocks'.format(np.round(total_selling_price,2)))
print('You will pay around  {} this amount of tax, sorry'.format(np.round(total_tax_to_pay_approx ,2)))

fill 1TZ, form 2042 C with  15709.02588085749
fill 1UZ , form 2042 C with the rebate in euro 10207.19212534135
fill 3VG , form 2042 C with capital gain 0
fill 3VH , form 2042 C with capital loss 0 except if you have other sources of capital losses. You fill 0 because you are removing these losses from the 1TZ field
You sold for 25916.22 this amount of stocks
You will pay around  7414.66 this amount of tax, sorry


# CSG deduction from previous year (2021)

If you ahve your previous tax filling, it should be equal to your 1TZ case of last year times the deductible CSG, which is at 6.8%. If you don't, you can use this, should work. It is very likely to be pre filled though. Still there in case of need

In [11]:
if vesting_gain_before_rebate_and_moins_value_2021 ==0:
    print('sorry, no CSG to be deducted for this year!')
    vesting_gain_after_rebate_and_moins_value_2021 =0
    deductible_csg = vesting_gain_after_rebate_and_moins_value_2021 * tax_info_dict['deductible_csg']
else:
    if Capital_gain_or_loss_2021 >0:
        vesting_gain_after_rebate_and_moins_value_2021 = vesting_gain_before_rebate_and_moins_value_2021 - rebate_amount_2021
        deductible_csg = vesting_gain_after_rebate_and_moins_value_2021 * tax_info_dict['deductible_csg']
    else:
        rebate_percent_2021 = rebate_amount_2021 / vesting_gain_before_rebate_and_moins_value_2021
        vesting_gain_after_rebate_and_moins_value_2021 = (vesting_gain_before_rebate_and_moins_value_2021 + Capital_gain_or_loss_2021) * (1-rebate_percent_2021)
        deductible_csg = vesting_gain_after_rebate_and_moins_value_2021 * tax_info_dict['deductible_csg']

In [12]:
print('fill box 6DE', deductible_csg)

fill box 6DE 277.08647335927674
